# SageMaker で Neural Network Libraries のコンテナを作成して学習する

#### ノートブックに含まれる内容

- [Newral Network Libraries](https://github.com/sony/nnabla) を使った学習用コンテナの作成
- SageMaker で BYOA(Bring Your Own Container) により，作成したコンテナを使って学習

#### ノートブックで使われている手法の詳細

- Docker
- MNIST ([nnabla-example](https://github.com/sony/nnabla-examples/blob/master/)の `mnist-collection/classification.py` を実行する形)

## 前準備

このノートブックでは Amazon Elastic Container Registory (ECR) を使用するため，あらかじめ SageMaker を実行している IAM Role に対して，以下の IAM Policy をアタッチしてください

```
AmazonEC2ContainerRegistryFullAccess
```

## 概要

SageMaker を使うことで，このような問題を解決することができます．SageMaker は Docker コンテナを活用することにより，モデル学習や API による推論をスケーラブルな形で実行します．そのため，SageMaker を実際に使用する前に，学習および推論を行うための Docker イメージをまず最初に作成します．それから，作成した Docker イメージを使って，実際に SageMaker API 経由で学習，および推論を実行します．

このノートブックでは，nnabla での学習・推論を行うための Docker イメージについて説明します．以下，まず Part 1 で Docker イメージのアーキテクチャについて説明，Part 2 で実際に Docker イメージの作成を行います．

## SageMaker 用の Docker イメージの構成

### SageMaker での Docker の利用の仕方
SageMaker の Docker イメージは，学習のときと推論のときで，同じイメージを用いることができます．SageMaker の中では，学習と推論のそれぞれでコンテナを立ち上げる際に，以下のコマンドが実行されます

* 学習: `docker run $IMAGE_ID train`

* 推論: `docker run $IMAGE_ID serve`

このため，Docker イメージは `train` および `serve` というコマンドを持つ必要があります．この例では，Docker イメージ作成時に使用するスクリプト群をまとめて `container` ディレクトリに以下のように配置しました．`container/mnist-collection` 内に `train` のスクリプトが配置されているのが確認できるかと思います（この例では，学習のみで推論は行わないため，serve スクリプトは用意していません）．このスクリプトは Bash で書かれていますが，実際にはどの言語で書いても問題はありません．

    .
    └── container
        ├── Dockerfile
        ├── build_and_push.sh
        └── mnist-collection
            ├── args.py
            ├── classification.py
            ├── mnist_data.py
            ├── requirements.txt
            └── train


* __`Dockerfile`__ には，Docker イメージをどのようにビルドするかが記述されています
* __`build_and_push.sh`__ は Dockerfile を使ってコンテナイメージをビルドし，ECR にプッシュするためのスクリプトです
* __`mnist-collection`__ コンテナ内に含まれるファイルを配置したディレクトリです


In [ ]:
# ディレクトリの中身の確認
!ls -lR container

In [ ]:
# train スクリプトの中身の確認
!cat container/mnist-collection/train

### 学習時のコンテナの実行

SageMaker が学習ジョブを走らせる際，`train` スクリプトが通常の Python プログラムのように実行されます．その際に SageMaker の仕様として，コンテナ内の `/opt/ml` ディレクトリ内に，さまざまなファイルを配置して使用する形をとります．

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

#### インプット

* `/opt/ml/input/config` には，どのように学習処理を実行するかの情報が置かれます．`hyperparameters.json` はハイパーパラメタの名前とその値を JSON フォーマットで格納したファイルです．値は常に `string` 型として読みだされるため，その後適切な型に変換する必要があります．`resourceConfig.json` はマルチノードでの分散学習を行う際のネットワークレイアウトを記述した JSON フォーマットのファイルです．
* `/opt/ml/input/data/<channel_name>/` はデータ入力方式が FILE モードのときに使われるディレクトリです．チャンネルはジョブ実行時に叩く `CreateTrainingJob` に引き渡すパラメタとして指定することができます．入力データはチャネルごとに，こちらもパラメタで指定された S3 ディレクトリからロードされたものが配置されます．今回の学習では，入力データはコード内で Web からダウンロードしてくるため，ここには何も置かれません
* `/opt/ml/input/data/<channel_name>_<epoch_number>` はデータ入力方式が PIPE モードのときに使われるディレクトリです．エポックは 0 から始まり順に増えていきます．ディレクトリ名はチャンネルとエポックで指定されます

#### アウトプット

* `/opt/ml/model/` は，アルゴリズムにより生成された結果のモデルが保存されるディレクトリです．モデルのフォーマットは自由に指定することができます．単一ファイルでもよいですし，階層構造を持ったディレクトリの形でも構いません．SageMaker はこのディレクトリ内のすべてのデータを圧縮済みの tar アーカイブにまとめます．このアーカイブファイルは，`DescribeTrainingJob` API のレスポンスに含まれる S3 ロケーションに置かれます
* `/opt/ml/output` にはジョブが失敗した際に，その原因が記述された `failure` ファイルが配置されます．このファイルの中身は，`DescribeTrainingJob` API のレスポンスに含まれる `FailureReason` の内容と同じです．ジョブが成功した際には，ここには何も書き出されません

### 推論時のコンテナの実行

推論時には，コンテナが API サーバとしてホストされた形で実行されます．そのため，HTTP 経由で推論のリクエストを受け付けることができます．SageMaker で API サーバをホストする際には，以下の 2 つのエンドポイントが必要です

* `/ping` はインフラからの `GET` リクエストを受けるためのエンドポイントです．リクエストを受けたら，レスポンスコード 200 を返します
 
* `/invocations` はクライアントからの `POST` 推論リクエストを受けるためのエンドポイントです．リクエストとレスポンスのフォーマットは自由に指定することができます．クライアントで `ContentType` と `Accept` ヘッダをつけた場合には，そのままエンドポイント側に引き渡されます

推論用のコンテナでは，SageMaker はモデルファイルを学習時と同じディレクトリに配置して使用します

    /opt/ml
    └── model
        └── <model files>

## Docker イメージの作成

### Dockerfile

ここまで説明してきた仕組みを実現するために，Dockerfile でコンテナイメージの構成を定義します．こちらは，[nnabla-examples の Dockerfile](https://github.com/sony/nnabla-examples/blob/master/Dockerfile)を少しだけ加工したものになります．

In [ ]:
!cat container/Dockerfile

### コンテナイメージをビルドして登録

以下のシェルで，`docker build` コマンドを使ってコンテナイメージをビルドし，ECR (Elastic Container Registry) にプッシュします．このスクリプトは，`container/build-and-push.sh` にシェルスクリプトとしてまとまっており，`build-and-push.sh nnabla-example-mnist` の形で実行することで，`nnabla-example-mnist` イメージを ECR にプッシュすることができます．

ECR リポジトリは，SageMaker のノートブックインスタンスがあるのと同一リージョンのものが使われます．もしリポジトリがない場合には，自動的に作られます．

以下のスクリプトを実行する前に，**<span style="color: red;">5 行目の `account_number=XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
%%sh

# アルゴリズムの名前
# アカウントナンバーを修正
account_number=XX
algorithm_name=nnabla-example-mnist-$account_number

cd container
chmod +x mnist-collection/train

account=$(aws sts get-caller-identity --query Account --output text)

# 現在の設定を確認して，リージョンをセット (もし定義されていない場合には，us-west-2 に設定)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# もしリポジトリが ECR に存在しない場合には作成

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# ECR からログインコマンドを取得し，直接実行
$(aws ecr get-login --region ${region} --no-include-email)

docker build  -t ${algorithm_name} . --build-arg CUDA_VER=9.2 --build-arg CUDNN_VER=7 --build-arg PYTHON_VER=3.6
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

無事上記コマンドが実行できたら，ECR の URL を開いて，作成した `decision-trees-sample-XX` のリポジトリが存在することを確認．

## SageMaker セッションのセットアップ

In [ ]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

# AWS credential で指定された role を返す
role = get_execution_role()

# SageMaker のセッションを作成
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## モデルの学習を実行

SageMaker で学習を行うために，SageMaker SDK で Estimator オブジェクトをつくります．このオブジェクトには，学習をおこなうために以下の設定が含まれます．その上で，fit() メソッドで学習を実施します．学習には 5 分程度時間がかかります．

- container name: 上で作成した ECR のコンテナイメージ
- role: ジョブを実行する IAM role
- instance count: 学習ジョブに使うインスタンス数
- instance type 学習ジョブに使うインスタンスタイプ
- output path: 学習の成果物が置かれる S3 の場所
- session: すぐ上で作成した，SageMaker セッション
- また，以下を実行する前に，3 行目の **<span style="color: red;">5 行目の `account_number=XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/nnabla-example-mnist-XX'.format(account, region)

classifier = sage.estimator.Estimator(
    image,
    role,
    1,
    'ml.m4.xlarge',
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess)

classifier.fit()

無事実行できたら，コンソールに戻って，ジョブ実行画面を開いてください．学習済みモデルが出力されている S3 パスを開いて，実際にモデルが作成されたことを確認してください．以上で独自コンテナの作成と実行が完了しました．